In [ ]:
import sys
sys.path.insert(0, '/kaggle/input/kb-l5kit/l5kit/')

In [ ]:
import os
from pprint import pprint
from typing import Dict

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision.models.mobilenet import mobilenet_v2
from tqdm.notebook import tqdm
import wandb

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv

## Prepare Data path and load cfg
By setting the L5KIT_DATA_FOLDER variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles/"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("/kaggle/input/lyft-config-files/agent_motion_config.yaml")

In [ ]:
cfg['model_params']['history_num_frames'] = 10
cfg['train_params']['max_num_steps'] = 25000
cfg['train_params']['checkpoint_every_n_steps'] = 10000
cfg['train_data_loader']['batch_size'] = 12
cfg['train_data_loader']['num_workers'] = 4
cfg['train_data_loader']['key'] = 'scenes/train.zarr'
pprint(cfg)

## Model

We will be building MobileNetV2 here and concatenating agent states (velocity, acceleration and heading offsets). We must replace the input and the final layer to address our requirements.

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x
    
class MobilenetV2LSTM(nn.Module):
    def __init__(self, config: Dict):
        super(MobilenetV2LSTM, self).__init__()
        self.cfg = config
        self.batch_size = self.cfg['train_data_loader']['batch_size']
        self.hist_frames = self.cfg['model_params']['history_num_frames']
        self.fc_infeatures = 1280 + (2 * (self.hist_frames + 1)) + (2 * self.hist_frames) + (self.hist_frames + 1)
        self.num_targets = 2 * self.cfg["model_params"]["future_num_frames"]
#         self.seq_len = 1
#         self.input_size = 128
#         self.hidden_size = 128
        self.cnn = self.build_basecnn()
        self.fc1 = nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(in_features=self.fc_infeatures, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(in_features=4096, out_features=self.num_targets)
        )
#         self.lstm = nn.LSTM(
#             input_size=self.input_size,
#             hidden_size=self.hidden_size,
#             num_layers=1,
#             batch_first=True
#         )
#         self.fc2 = nn.Linear(in_features=128, out_features=100)
#         self.hidden_cell = (torch.zeros(self.batch_size, 1, self.hidden_size),
#                             torch.zeros(self.batch_size, 1, self.hidden_size))

    def forward(self, x, vel, accel, yaw):
        x = self.cnn(x)
        vel = vel.reshape(self.batch_size, -1)
        accel = accel.reshape(self.batch_size, -1)
        yaw = yaw.reshape(self.batch_size, -1)
        x = torch.cat([x, vel, accel, yaw], dim=1)
        x = self.fc1(x)
#         cnn_out = self.fc1(x)
#         lstm_in = cnn_out.view(self.batch_size, self.seq_len, self.input_size)
#         lstm_out, self.hidden_cell = self.lstm(lstm_in)
#         fc_in = lstm_out.view(self.batch_size, lstm_out.shape[2])
#         x = self.fc2(fc_in)

        return x

    def build_basecnn(self):
        # change input channels number to match the rasterizer's output
        mnet = mobilenet_v2(pretrained=True)
        num_history_channels = (self.cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels
        mnet.features[0][0] = nn.Conv2d(
            num_in_channels,
            mnet.features[0][0].out_channels,
            kernel_size=mnet.features[0][0].kernel_size,
            stride=mnet.features[0][0].stride,
            padding=mnet.features[0][0].padding,
            bias=False,
        )

        mnet.classifier = Identity()
        
        return mnet

In [ ]:
# def build_model(cfg: Dict) -> torch.nn.Module:
#     # load pre-trained Conv2D model
#     model = mobilenet_v2(pretrained=True)

#     # change input channels number to match the rasterizer's output
#     num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
#     num_in_channels = 3 + num_history_channels
#     model.features[0][0] = nn.Conv2d(
#         num_in_channels,
#         model.features[0][0].out_channels,
#         kernel_size=model.features[0][0].kernel_size,
#         stride=model.features[0][0].stride,
#         padding=model.features[0][0].padding,
#         bias=False,
#     )
#     # change output size to (X, Y) * number of future states
#     num_targets = 2 * cfg["model_params"]["future_num_frames"]
#     model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=num_targets)

#     return model

In [ ]:
def forward(data, model, device, criterion):
    im_inputs = data["image"].to(device)
    vel_inputs = data["history_velocities"].to(device)
    accel_inputs = data["history_accels"][:, :-1, :].to(device)  # removing last history frame since we don't have accel for it
    yaw_inputs = data["history_yaws"].to(device)

    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    outputs = model(im_inputs, vel_inputs, accel_inputs, yaw_inputs).reshape(targets.shape)
    loss = criterion(outputs, targets)
    # not all the output steps are valid, but we can filter them out from the loss using availabilities
    loss = loss * target_availabilities
    loss = loss.mean()
    return loss, outputs

## Load the Train Data

Our data pipeline map a raw `.zarr` folder into a multi-processing instance ready for training by:
- loading the `zarr` into a `ChunkedDataset` object. This object has a reference to the different arrays into the zarr (e.g. agents and traffic lights);
- wrapping the `ChunkedDataset` into an `AgentDataset`, which inherits from torch `Dataset` class;
- passing the `AgentDataset` into a torch `DataLoader`

In [ ]:
rasterizer = build_rasterizer(cfg, dm)
train_cfg = cfg["train_data_loader"]
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)

# create training splits for transfer learning --- wait for pytorch update on kaggle kernels
# train1_size = 25000
# trainrem_size = len(train_dataset) - train1_size
# train_dataset, _ = random_split(train_dataset, [train1_size, trainrem_size], generator=torch.Generator().manual_seed(42))
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])
print(train_dataset)
print(len(train_dataset))
print(len(train_dataloader))

In [ ]:
# ==== INIT MODEL
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MobilenetV2LSTM(cfg).to(device)
# model = build_model(cfg).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss(reduction="none")

## Training

In [ ]:
def train(model, device, train_loader, criterion, optimizer, max_steps, checkpoint_steps):
    # Switch model to training mode. This is necessary for layers like dropout, batchnorm etc 
    # which behave differently in training and evaluation mode
    model.train()
    losses_train = []
    step = 1
    
    # We loop over the data iterator, and feed the inputs to the network and adjust the weights.
    train_pbar = tqdm(enumerate(train_loader), desc="Training steps", leave=True, total=max_steps)
    for batch_idx, data in train_pbar:
        # Forward pass to calculate loss
        loss, _ = forward(data, model, device, criterion)

        # Reset the gradients to 0 for all learnable weight parameters
        optimizer.zero_grad()

        # Backward pass: compute the gradients of the loss w.r.t. the model's parameters
        loss.backward()

        # Update the model weights
        optimizer.step()

        # Get average loss of iterations so far
        losses_train.append(loss.item())
        avg_train_loss = np.mean(losses_train)

        # wandb logging - metrics to track
        wandb.log({'Training Loss': loss.item(), 'Avg Training Loss': avg_train_loss})

        train_pbar.set_description(f" Avg train loss: {avg_train_loss}")
        
        if step % checkpoint_steps == 0 and step != max_steps:
            torch.save(model.state_dict(), f'l5run3_iter{step}_mobilenetv2.pth')
        
        if step >= max_steps:
            return avg_train_loss
        step += 1

    return avg_train_loss

In [ ]:
# Main training and validation loop

# Initialize wandb
wandb.init(project='lyft-competition', name='mobilenetv2', anonymous='allow')

# WandB – Config is a variable that holds and saves hyperparameters and inputs
wandb_cfg = wandb.config                                         # initialize wandb config
wandb_cfg.batch_size = cfg['train_data_loader']['batch_size']    # input batch size for training (default: 64)
wandb_cfg.test_batch_size = cfg['val_data_loader']['batch_size'] # input batch size for testing (default: 1000)
wandb_cfg.steps = cfg['train_params']['max_num_steps']
wandb_cfg.epochs = 1                                             # number of epochs to train (default: 10)
wandb_cfg.lr = 1e-3                                              # learning rate (default: 0.01)
wandb_cfg.seed = 42                                              # random seed (default: 42)
wandb_cfg.log_interval = 0                                       # how many batches to wait before logging training status

# Set random seeds and deterministic pytorch for reproducibility
# random.seed(wandb_cfg.seed)       # python random seed
# torch.manual_seed(wandb_cfg.seed) # pytorch random seed
# np.random.seed(wandb_cfg.seed)    # numpy random seed
# torch.backends.cudnn.deterministic = True

# WandB – wandb.watch() automatically fetches all layer dimensions, gradients, model parameters and logs them automatically to your dashboard.
# Using log="all" log histograms of parameter values in addition to gradients
# wandb.watch(model)

# pbar = tqdm(range(1, wandb_cfg.epochs+1), desc="Epochs")
# for epoch in pbar:
#     train_loss = train(model, device, train_dataloader, criterion, optimizer, epoch)
#     val_loss = val(model, device, val_dataloader, criterion, epoch)
#     print(f"Epoch: {epoch}, Train loss: {train_loss}, Val loss: {val_loss}")

checkpoint_steps =  cfg['train_params']['checkpoint_every_n_steps']   
train_loss = train(model, device, train_dataloader, criterion, optimizer, wandb_cfg.steps, checkpoint_steps)

In [ ]:
# Save model
torch.save(model.state_dict(), 'l5run3_mobilenetv2.pth')